In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [120]:
train = pd.read_csv("train.csv")
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [121]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


# 결측치 예측을 위한 데이터 전처리

## 필요없어 보이는 feature 삭제
PassengerId, Name 삭제

In [122]:
train.drop(columns=["PassengerId","Name"], inplace=True)
train

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False
8689,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,False
8690,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,True
8691,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False


## Cabin 나눠서 표현하기
Cabin을 확인해보면 총 3부분으로 이루어져 있으므로 이를 분석하기 쉽게 3부분으로 쪼개어 표현해 놓는다.

In [123]:
train[["Cabin1","Cabin2", "Cabin3"]] = train["Cabin"].str.split("/", expand=True)
train
# 새롭게 추가된 Cabin1, Cabin2, Cabin3 feature들을 확인할 수 있다.

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin2,Cabin3
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,B,0,P
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,F,0,S
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,A,0,S
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,A,0,S
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,F,1,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False,A,98,P
8689,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,False,G,1499,S
8690,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,True,G,1500,S
8691,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False,E,608,S


In [124]:
# Cabin은 삭제해준다.
train.drop(columns="Cabin", inplace=True)
train

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin2,Cabin3
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,B,0,P
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,F,0,S
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,A,0,S
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,A,0,S
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,F,1,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False,A,98,P
8689,Earth,True,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,False,G,1499,S
8690,Earth,False,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,True,G,1500,S
8691,Europa,False,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False,E,608,S


## 범주형 label encoding하기
HomePlanet, CryoSleep, Destination, VIP, Cabin1, Cabin3 범주형들을 인코딩

In [125]:
from sklearn.preprocessing import LabelEncoder

le_HomePlanet = LabelEncoder()
train["HomePlanet"] = le_HomePlanet.fit_transform(train["HomePlanet"])

le_CryoSleep = LabelEncoder()
train["CryoSleep"] = le_CryoSleep.fit_transform(train["CryoSleep"])

le_Destination = LabelEncoder()
train["Destination"] = le_Destination.fit_transform(train["Destination"])

le_VIP = LabelEncoder()
train["VIP"] = le_VIP.fit_transform(train["VIP"])

le_Cabin1 = LabelEncoder()
train["Cabin1"] = le_Cabin1.fit_transform(train["Cabin1"])

le_Cabin3 = LabelEncoder()
train["Cabin3"] = le_Cabin3.fit_transform(train["Cabin3"])

In [126]:
train

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin2,Cabin3
0,1,0,2,39.0,0,0.0,0.0,0.0,0.0,0.0,False,1,0,0
1,0,0,2,24.0,0,109.0,9.0,25.0,549.0,44.0,True,5,0,1
2,1,0,2,58.0,1,43.0,3576.0,0.0,6715.0,49.0,False,0,0,1
3,1,0,2,33.0,0,0.0,1283.0,371.0,3329.0,193.0,False,0,0,1
4,0,0,2,16.0,0,303.0,70.0,151.0,565.0,2.0,True,5,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,1,0,0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,False,0,98,0
8689,0,1,1,18.0,0,0.0,0.0,0.0,0.0,0.0,False,6,1499,1
8690,0,0,2,26.0,0,0.0,0.0,1872.0,1.0,0.0,True,6,1500,1
8691,1,0,0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,False,4,608,1


## 결측치 데이터 뽑기

### HomePlanet 결측치 데이터

In [127]:
le_HomePlanet.classes_

array(['Earth', 'Europa', 'Mars', nan], dtype=object)

In [128]:
HomePlanet_Nan = train[train["HomePlanet"]==3]
HomePlanet_Nan

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin2,Cabin3
59,3,1,2,33.0,0,0.0,0.0,NaN,0.0,0.0,True,4,3,1
113,3,0,2,39.0,0,0.0,2344.0,0.0,65.0,6898.0,False,0,0,0
186,3,1,0,24.0,0,0.0,0.0,NaN,0.0,0.0,True,3,6,0
225,3,0,2,18.0,0,313.0,1.0,691.0,283.0,0.0,False,5,46,1
234,3,1,0,54.0,0,0.0,0.0,0.0,0.0,0.0,True,2,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8515,3,0,2,25.0,0,1258.0,0.0,22.0,19.0,0.0,False,4,582,0
8613,3,0,0,53.0,0,0.0,4017.0,0.0,13.0,3147.0,False,4,603,1
8666,3,0,0,38.0,2,28.0,1208.0,973.0,207.0,0.0,True,5,1792,1
8674,3,0,2,13.0,0,39.0,0.0,1085.0,24.0,0.0,False,5,1892,0


In [133]:
le_CryoSleep.classes_

array([False, True, nan], dtype=object)

In [134]:
CryoSleep_Nan = train[train["CryoSleep"]==2]
CryoSleep_Nan

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin2,Cabin3
92,0,2,2,2.0,0,0.0,0.0,0.0,0.0,0.0,True,6,12,0
98,0,2,2,27.0,0,0.0,0.0,570.0,2.0,131.0,False,5,21,0
104,1,2,2,40.0,0,0.0,331.0,0.0,0.0,1687.0,False,1,5,0
111,2,2,2,26.0,0,0.0,0.0,0.0,0.0,NaN,True,5,24,0
152,0,2,2,58.0,0,0.0,985.0,0.0,5.0,0.0,True,4,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8620,1,2,0,44.0,0,0.0,0.0,0.0,0.0,0.0,True,2,308,0
8651,0,2,2,8.0,0,0.0,0.0,0.0,0.0,0.0,False,6,1498,0
8664,0,2,2,32.0,0,0.0,0.0,0.0,0.0,0.0,True,6,1490,1
8675,0,2,2,44.0,0,1030.0,1015.0,0.0,11.0,NaN,True,5,1893,0


In [135]:
le_Destination.classes_

array(['55 Cancri e', 'PSO J318.5-22', 'TRAPPIST-1e', nan], dtype=object)

In [136]:
Destination_Nan = train[train["Destination"]==3]
Destination_Nan

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin2,Cabin3
47,2,1,3,19.0,0,0.0,0.0,0.0,0.0,0.0,True,5,10,0
128,0,0,3,34.0,0,0.0,22.0,0.0,564.0,207.0,False,4,5,0
139,0,0,3,41.0,0,0.0,0.0,0.0,0.0,607.0,False,5,32,0
347,3,0,3,23.0,0,348.0,0.0,0.0,4.0,368.0,False,6,64,0
430,0,1,3,50.0,0,0.0,0.0,0.0,0.0,0.0,False,6,67,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8372,0,1,3,20.0,0,0.0,0.0,0.0,0.0,0.0,True,6,1453,0
8551,2,1,3,41.0,0,0.0,0.0,0.0,0.0,0.0,True,5,1765,1
8616,2,1,3,33.0,0,0.0,0.0,0.0,0.0,0.0,True,5,1779,1
8621,1,0,3,41.0,1,0.0,7964.0,0.0,3238.0,5839.0,False,2,308,0


In [137]:
le_VIP.classes_

array([False, True, nan], dtype=object)

In [138]:
VIP_Nan = train[train["VIP"]==2]
VIP_Nan

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin2,Cabin3
38,0,0,0,15.0,2,0.0,492.0,48.0,20.0,401.0,False,5,8,1
102,0,0,2,0.0,2,0.0,0.0,0.0,0.0,0.0,True,6,19,1
145,2,1,2,35.0,2,0.0,0.0,0.0,0.0,0.0,True,5,37,0
228,2,1,0,14.0,2,0.0,0.0,0.0,0.0,0.0,True,5,47,1
566,2,0,2,NaN,2,43.0,152.0,182.0,1.0,2005.0,False,3,24,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8494,0,1,2,0.0,2,0.0,0.0,NaN,0.0,0.0,True,6,1460,1
8512,0,0,1,16.0,2,0.0,0.0,761.0,0.0,0.0,False,5,1858,0
8542,0,1,0,55.0,2,0.0,0.0,0.0,0.0,0.0,False,6,1469,1
8630,1,1,2,52.0,2,0.0,0.0,0.0,0.0,0.0,True,1,300,0


In [139]:
RoomService_Nan = train[train["RoomService"].isna()]
RoomService_Nan

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin2,Cabin3
25,0,1,1,1.0,0,NaN,0.0,0.0,0.0,0.0,False,4,0,1
35,2,0,2,20.0,0,NaN,0.0,1750.0,990.0,0.0,True,5,9,0
83,0,1,2,26.0,0,NaN,0.0,0.0,0.0,0.0,True,6,16,1
132,2,0,2,31.0,0,NaN,0.0,97.0,0.0,0.0,False,5,30,0
170,2,0,2,23.0,0,NaN,0.0,8.0,1072.0,0.0,False,5,41,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8425,0,0,2,47.0,0,NaN,1.0,0.0,967.0,5.0,False,4,591,1
8450,0,1,2,58.0,0,NaN,0.0,0.0,0.0,0.0,True,6,1463,0
8525,0,0,2,21.0,0,NaN,0.0,496.0,430.0,0.0,False,5,1865,0
8534,2,0,2,28.0,0,NaN,0.0,0.0,0.0,0.0,False,3,290,0


In [141]:
FoodCourt_Nan = train[train["FoodCourt"].isna()]
FoodCourt_Nan

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin2,Cabin3
95,0,0,2,24.0,0,0.0,NaN,0.0,0.0,17.0,True,5,24,1
218,0,1,1,27.0,0,0.0,NaN,0.0,0.0,0.0,False,6,36,1
367,0,0,0,17.0,0,177.0,NaN,0.0,310.0,7.0,False,5,89,0
385,1,0,2,31.0,0,417.0,NaN,7185.0,562.0,792.0,True,4,26,0
456,1,1,0,27.0,0,0.0,NaN,0.0,0.0,0.0,True,8,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8396,0,0,2,27.0,0,1.0,NaN,0.0,20.0,948.0,False,5,1840,0
8422,0,1,0,15.0,0,0.0,NaN,0.0,0.0,0.0,True,5,1742,1
8460,1,0,2,60.0,0,0.0,NaN,947.0,668.0,1.0,True,3,288,0
8537,0,0,2,26.0,0,0.0,NaN,0.0,1.0,48.0,True,5,1868,0


In [142]:
ShoppingMall_Nan = train[train["ShoppingMall"].isna()]
ShoppingMall_Nan

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin2,Cabin3
10,1,1,2,34.0,0,0.0,0.0,NaN,0.0,0.0,True,1,1,0
16,2,0,0,27.0,0,1286.0,122.0,NaN,0.0,0.0,False,5,3,0
23,0,1,0,29.0,0,0.0,0.0,NaN,0.0,0.0,False,4,0,1
28,1,1,2,62.0,0,0.0,0.0,NaN,0.0,0.0,True,2,2,1
40,0,1,0,30.0,0,0.0,0.0,NaN,0.0,0.0,False,6,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8607,2,1,2,2.0,0,0.0,0.0,NaN,0.0,0.0,True,5,1878,0
8611,0,0,1,24.0,0,NaN,599.0,NaN,0.0,0.0,True,5,1880,0
8633,0,0,2,60.0,0,0.0,8.0,NaN,360.0,74.0,False,6,1486,1
8641,0,0,1,22.0,0,623.0,0.0,NaN,0.0,53.0,True,5,1886,0


In [143]:
Spa_Nan = train[train["Spa"].isna()]
Spa_Nan

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin2,Cabin3
48,0,0,0,35.0,0,790.0,0.0,0.0,NaN,0.0,False,4,1,1
143,0,0,2,57.0,0,50.0,1688.0,0.0,NaN,135.0,True,6,28,1
245,1,1,2,39.0,0,0.0,0.0,0.0,NaN,0.0,True,3,8,1
269,1,1,2,50.0,0,0.0,0.0,0.0,NaN,0.0,True,1,8,1
289,0,0,2,18.0,0,0.0,2.0,0.0,NaN,0.0,False,6,44,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8554,2,0,2,31.0,0,2575.0,1.0,2.0,NaN,0.0,False,3,291,0
8566,1,1,0,37.0,0,0.0,0.0,0.0,NaN,0.0,True,1,350,1
8569,2,1,2,19.0,0,0.0,0.0,0.0,NaN,0.0,True,5,1769,1
8623,1,0,0,30.0,0,1569.0,NaN,0.0,NaN,2252.0,False,2,308,0


In [144]:
VRDeck_Nan = train[train["VRDeck"].isna()]
VRDeck_Nan

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin2,Cabin3
7,0,1,2,28.0,0,0.0,0.0,0.0,0.0,NaN,True,6,0,1
111,2,2,2,26.0,0,0.0,0.0,0.0,0.0,NaN,True,5,24,0
115,2,1,2,24.0,0,0.0,0.0,0.0,0.0,NaN,True,5,25,0
251,0,0,2,35.0,0,0.0,0.0,0.0,888.0,NaN,False,8,NaN,2
288,1,1,2,20.0,0,0.0,0.0,0.0,0.0,NaN,True,1,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8404,0,0,2,19.0,0,0.0,810.0,21.0,788.0,NaN,False,4,590,1
8408,0,1,0,17.0,0,0.0,0.0,0.0,0.0,NaN,True,6,1448,1
8642,0,1,2,21.0,0,0.0,0.0,0.0,0.0,NaN,False,6,1497,0
8665,0,1,2,33.0,0,0.0,0.0,0.0,0.0,NaN,False,6,1491,1


In [145]:
le_Cabin1.classes_

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T', nan], dtype=object)

In [146]:
Cabin1_Nan = train[train["Cabin1"]==8]
Cabin1_Nan

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin2,Cabin3
15,0,0,2,31.0,0,32.0,0.0,876.0,0.0,0.0,False,8,NaN,2
93,2,1,2,31.0,0,0.0,0.0,0.0,0.0,0.0,True,8,NaN,2
103,1,0,2,32.0,0,0.0,410.0,6.0,3929.0,764.0,False,8,NaN,2
222,2,0,2,37.0,0,637.0,0.0,0.0,92.0,319.0,False,8,NaN,2
227,2,1,2,43.0,0,0.0,0.0,0.0,0.0,0.0,True,8,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8209,1,0,0,53.0,0,0.0,1127.0,0.0,3939.0,NaN,False,8,NaN,2
8475,1,0,0,36.0,1,132.0,3479.0,0.0,3786.0,0.0,False,8,NaN,2
8485,1,1,0,25.0,0,0.0,0.0,0.0,0.0,0.0,True,8,NaN,2
8509,0,1,2,1.0,0,0.0,0.0,0.0,0.0,0.0,False,8,NaN,2


In [149]:
Cabin2_Nan = train[train["Cabin2"].isna()]
Cabin2_Nan

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin2,Cabin3
15,0,0,2,31.0,0,32.0,0.0,876.0,0.0,0.0,False,8,NaN,2
93,2,1,2,31.0,0,0.0,0.0,0.0,0.0,0.0,True,8,NaN,2
103,1,0,2,32.0,0,0.0,410.0,6.0,3929.0,764.0,False,8,NaN,2
222,2,0,2,37.0,0,637.0,0.0,0.0,92.0,319.0,False,8,NaN,2
227,2,1,2,43.0,0,0.0,0.0,0.0,0.0,0.0,True,8,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8209,1,0,0,53.0,0,0.0,1127.0,0.0,3939.0,NaN,False,8,NaN,2
8475,1,0,0,36.0,1,132.0,3479.0,0.0,3786.0,0.0,False,8,NaN,2
8485,1,1,0,25.0,0,0.0,0.0,0.0,0.0,0.0,True,8,NaN,2
8509,0,1,2,1.0,0,0.0,0.0,0.0,0.0,0.0,False,8,NaN,2


In [147]:
le_Cabin3.classes_

array(['P', 'S', nan], dtype=object)

In [148]:
Cabin3_Nan = train[train["Cabin3"]==2]
Cabin3_Nan

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin2,Cabin3
15,0,0,2,31.0,0,32.0,0.0,876.0,0.0,0.0,False,8,NaN,2
93,2,1,2,31.0,0,0.0,0.0,0.0,0.0,0.0,True,8,NaN,2
103,1,0,2,32.0,0,0.0,410.0,6.0,3929.0,764.0,False,8,NaN,2
222,2,0,2,37.0,0,637.0,0.0,0.0,92.0,319.0,False,8,NaN,2
227,2,1,2,43.0,0,0.0,0.0,0.0,0.0,0.0,True,8,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8209,1,0,0,53.0,0,0.0,1127.0,0.0,3939.0,NaN,False,8,NaN,2
8475,1,0,0,36.0,1,132.0,3479.0,0.0,3786.0,0.0,False,8,NaN,2
8485,1,1,0,25.0,0,0.0,0.0,0.0,0.0,0.0,True,8,NaN,2
8509,0,1,2,1.0,0,0.0,0.0,0.0,0.0,0.0,False,8,NaN,2


# 각 feature를 label하여 학습

In [41]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6606 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    6606 non-null   int32  
 1   CryoSleep     6606 non-null   int32  
 2   Destination   6606 non-null   int32  
 3   Age           6606 non-null   float64
 4   VIP           6606 non-null   int32  
 5   RoomService   6606 non-null   float64
 6   FoodCourt     6606 non-null   float64
 7   ShoppingMall  6606 non-null   float64
 8   Spa           6606 non-null   float64
 9   VRDeck        6606 non-null   float64
 10  Transported   6606 non-null   bool   
 11  Cabin1        6606 non-null   int32  
 12  Cabin2        6606 non-null   object 
 13  Cabin3        6606 non-null   int32  
dtypes: bool(1), float64(6), int32(6), object(1)
memory usage: 574.2+ KB


In [42]:
print(le_HomePlanet.classes_)
print(le_CryoSleep.classes_)
print(le_Destination.classes_)
print(le_VIP.classes_)
print(le_Cabin1.classes_)
print(le_Cabin3.classes_)

['Earth' 'Europa' 'Mars']
[False True]
['55 Cancri e' 'PSO J318.5-22' 'TRAPPIST-1e']
[False True]
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'T']
['P' 'S']


학습을 위한 전처리는 마무리되었으므로, 학습 실행

## 모델 : 범주형( DecisionTreeClassifier, RandomForestClassifier )

In [45]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

tree = DecisionTreeClassifier(random_state=0)
forest = RandomForestClassifier(random_state=0)

params ={
    "max_depth":[3, 6, 9, 12, 15]
}

gs_tree = GridSearchCV(tree,
                      param_grid=params,
                      scoring="accuracy",
                      cv=4,
                      n_jobs=-1)
gs_forest = GridSearchCV(forest,
                      param_grid=params,
                      scoring="accuracy",
                      cv=4,
                      n_jobs=-1)

### HomePlanet

In [47]:
X_train = train.drop(columns="HomePlanet")
y_train = train["HomePlanet"]
gs_tree.fit(X_train, y_train)
gs_forest.fit(X_train, y_train)

GridSearchCV(cv=4, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [3, 6, 9, 12, 15]}, scoring='accuracy')

In [51]:
gs_tree.best_score_, gs_tree.best_params_

(0.9223423546958847, {'max_depth': 12})